In [1]:
# Random Forest Grid classification method
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
print(np.__version__)

2.2.4


In [3]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset.head(133)

User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
128  15722758    Male   30            17000          0
129  15792102  Female   26            84000          0
130  15675185    Male   31            58000          0
131  15801247    Male   33            31000          0
132  15725660    Male   30            87000          0

[133 rows x 5 columns]

In [5]:
dataset

User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]

In [6]:
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [7]:
dataset

User ID  Age  EstimatedSalary  Purchased  Gender_Male
0    15624510   19            19000          0            1
1    15810944   35            20000          0            1
2    15668575   26            43000          0            0
3    15603246   27            57000          0            0
4    15804002   19            76000          0            1
..        ...  ...              ...        ...          ...
395  15691863   46            41000          1            0
396  15706071   51            23000          1            1
397  15654296   50            20000          1            0
398  15755018   36            33000          0            1
399  15594041   49            36000          1            0

[400 rows x 5 columns]

In [8]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [9]:
#split into training set and test
# In regression, data splitting is not required. Whereas in Classification it is required
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [10]:
# If the predicted values is between the defined range, then standard sccalar is required
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [11]:
# Import the Random Forest classifier model
from sklearn.ensemble import RandomForestClassifier

In [12]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [13]:
# Loading the GridsearchCV 
from sklearn.model_selection import GridSearchCV

# configuring the parameters specific to this model
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 


# Scoring is important in this step. This is equal to R_Square in the regression
# This parameter is suitable for both imbalanced and balanced data
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
# In place of f1_weighted, other parameters listed in the above page can be replaced and tested
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
   
# fitting the model for grid search. Model gets created in this step
grid.fit(X_train_, y_train) 


Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\rhenaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
17 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\rhenaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rhenaj\AppData\Local\Packages\PythonSoftwareFound

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [14]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test_) 
   
# Importing the confusion matrix object
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)


# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [15]:
print(grid.best_params_)

{'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 10}


In [16]:
table=pd.DataFrame.from_dict(re)

In [17]:
table

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.002145      0.000473         0.000000        0.000000   
1        0.001869      0.000546         0.000000        0.000000   
2        0.039299      0.003885         0.007653        0.001518   
3        0.425476      0.016935         0.023236        0.001469   
4        0.046664      0.009266         0.007283        0.000895   
5        0.397661      0.019560         0.019457        0.002961   
6        0.001256      0.000156         0.000000        0.000000   
7        0.001270      0.000269         0.000000        0.000000   
8        0.049199      0.010696         0.010981        0.006022   
9        0.366668      0.014189         0.021494        0.001739   
10       0.042408      0.007380         0.009415        0.002734   
11       0.440498      0.031932         0.028113        0.006369   

   param_criterion param_max_features  param_n_estimators  \
0             gini               auto                  10   
1             gini               auto                 100   
2             gini               sqrt                  10   
3             gini               sqrt                 100   
4             gini               log2                  10   
5             gini               log2                 100   
6          entropy               auto                  10   
7          entropy               auto                 100   
8          entropy               sqrt                  10   
9          entropy               sqrt                 100   
10         entropy               log2                  10   
11         entropy               log2                 100   

                                               params  split0_test_score  \
0   {'criterion': 'gini', 'max_features': 'auto', ...                NaN   
1   {'criterion': 'gini', 'max_features': 'auto', ...                NaN   
2   {'criterion': 'gini', 'max_features': 'sqrt', ...           0.687500   
3   {'criterion': 'gini', 'max_features': 'sqrt', ...           0.727273   
4   {'criterion': 'gini', 'max_features': 'log2', ...           0.687500   
5   {'criterion': 'gini', 'max_features': 'log2', ...           0.764706   
6   {'criterion': 'entropy', 'max_features': 'auto...                NaN   
7   {'criterion': 'entropy', 'max_features': 'auto...                NaN   
8   {'criterion': 'entropy', 'max_features': 'sqrt...           0.833333   
9   {'criterion': 'entropy', 'max_features': 'sqrt...           0.764706   
10  {'criterion': 'entropy', 'max_features': 'log2...           0.764706   
11  {'criterion': 'entropy', 'max_features': 'log2...           0.764706   

    split1_test_score  split2_test_score  split3_test_score  \
0                 NaN                NaN                NaN   
1                 NaN                NaN                NaN   
2            0.842105           0.809524           0.871795   
3            0.842105           0.809524           0.900000   
4            0.810811           0.809524           0.900000   
5            0.842105           0.809524           0.900000   
6                 NaN                NaN                NaN   
7                 NaN                NaN                NaN   
8            0.810811           0.837209           0.871795   
9            0.871795           0.809524           0.900000   
10           0.789474           0.761905           0.894737   
11           0.842105           0.809524           0.900000   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
0                 NaN              NaN             NaN                9  
1                 NaN              NaN             NaN                9  
2            0.894737         0.821132        0.072686                8  
3            0.923077         0.840396        0.069486                5  
4            0.944444         0.830456        0.088420                6  
5            0.923077         0.847882        0.057954                4 

In [18]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 10}: 0.9180821729616219


In [19]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 5 44]]


In [20]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        85
           1       0.88      0.90      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.91      0.91       134
weighted avg       0.92      0.92      0.92       134



In [21]:
table_pr=pd.DataFrame()

In [22]:
print(X_test)

     Age  EstimatedSalary  Gender_Male
132   30            87000            1
309   38            50000            0
341   35            75000            1
196   30            79000            0
246   35            50000            0
..   ...              ...          ...
168   29           148000            1
150   26            15000            0
393   60            42000            1
66    24            19000            1
240   42           149000            1

[134 rows x 3 columns]


In [23]:
table_pr['Age']=X_test["Age"]
table_pr['Gender_male']=X_test["Gender_Male"]
table_pr['Estimated_Salary']=X_test["EstimatedSalary"]
table_pr['Actual']=y_test
table_pr['Predicted']=grid_predictions

In [24]:
table_pr.head(134)

Age  Gender_male  Estimated_Salary  Actual  Predicted
132   30            1             87000       0          0
309   38            0             50000       0          0
341   35            1             75000       0          0
196   30            0             79000       0          0
246   35            0             50000       0          0
..   ...          ...               ...     ...        ...
168   29            1            148000       1          1
150   26            0             15000       0          0
393   60            1             42000       1          1
66    24            1             19000       0          0
240   42            1            149000       1          1

[134 rows x 5 columns]

In [25]:
table_pr.to_csv("cm.csv",index=False)

In [26]:
dataset['']
dataset['Predicted']=grid_predictions

KeyError: ''

In [ ]:
age_input=float(input("Age:"))
salary_input=float(input("BMI:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

In [ ]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))